<a href="https://colab.research.google.com/github/Bewthanapon/Speech-to-Text/blob/main/1_Speech_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar  6 18:27:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Install & Import Library

In [7]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 40.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/

In [20]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 40.3 MB/s eta 0:00:00


In [24]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.2 MB/s eta 0:00:00


- whisper: OpenAI's automatic speech recognition (ASR) model.
- librosa: A library for analyzing and processing audio files.
- soundfile: Handles reading and writing audio files.
- jiwer: A library for calculating error rates in speech-to-text outputs (WER, CER).
- pythainlp.tokenize: Tokenization for Thai language processing.

In [27]:
import whisper
import librosa
import soundfile as sf
from jiwer import wer, cer
from pythainlp.tokenize import word_tokenize

In [2]:
!gdown 12mWRq6bATH8A0SDFii2z3fqVKcEADCpW

Downloading...
From: https://drive.google.com/uc?id=12mWRq6bATH8A0SDFii2z3fqVKcEADCpW
To: /content/Telesales Call Recording 2025-02-25 08_23_26.m4a
100% 8.69M/8.69M [00:00<00:00, 120MB/s]


# __Reference_text__

In [3]:
ref_text = "สวัสดีค่ะ ขอเรียนสายคุณโชคดี มีเงิน ค่ะดิฉัน นางสาวขายดี ทวีทรัพย์ ติดต่อจากบริษัทคุ้มดีจำกัดในเครือธนาคาร Fin4uทางบริษัทได้จัดทำโครงการพิเศษ ชื่อว่า คุ้มดีมีคืนสะดวกเรียนสายได้สักครู่ไหมคะ สะดวกค่ะ ขอบพระคุณค่ะ หนูขออนุญาตบันทึกสายสนทนานะคะดิฉัน นางสาวขายดี ทวีทรัพย์ เลขที่ใบอนุญาตตัวแทน ตท1902/2025 ค่ะ ทางโครงการขอเรียนเชิญคุณลูกค้าเข้าร่วมโครงการ คุ้มดีมีคืนค่ะซึ่งเป็นโครงการออมทรัพย์พิเศษในรูปแบบการประกันชีวิตสะสมทรัพย์ซึ่งผลประโยชน์ที่ลูกค้าจะได้รับแตกต่างจากที่อื่นๆเพราะการันตีผลตอบแทนรวมตลอดสัญญาสูงสุดถึง 616% ของทุนประกันชีวิตเลยค่ะโดยคุณลูกค้าไม่ต้องใช้เงินก้อนเป็นหมื่นเป็นแสนเลยนะคะเพียงออมเงินทุกๆเดือนเท่าๆกันตัวอย่างเช่น คุณลูกค้าจ่ายเบี้ยเพียงวันละ 90 บาทหรือรายเดือนเดือนละ 2,700 บาทคุณลูกค้าจะได้เงินคืนทุกปีตั้งแต่ปีที่ 1 จนถึงปีที่ 14การันตีรับเงินคืนปีละ 3,200 บาท 14 ปีเท่ากับได้ออมเพิ่มขึ้น 44,800 บาทสิ้นปีที่ 15-24รับเงินการันตีเพิ่มขึ้นไปอีกเป็นปีละ 6,400 บาทรวม 10 ปีเป็นเงินเพิ่ม 64,000 บาทค่ะและเมื่อสิ้นปีที่ 24 ที่ครบอายุสัญญาคุณลูกค้าจะได้เงินออม 384,000 บาทเมื่อรวมกับเงินคืนที่ได้ไปในปีที่ 1 ถึงปีที่ 24แล้วเป็นเงิน 492,800 บาทเลยทีเดียวค่ะที่สำคัญคุณลูกค้าออมเงินเพียงวันละเพียงแค่ 14 ปีแรกแต่มีเงินเพิ่มทุกๆปีถึง 24 ปีเลยค่ะนอกจากนี้ลูกค้ายังสามารถนำยอดเบี้ยที่ชำระในปีที่ 14ปีแรกไปลดหย่อนภาษีในแต่ละปีและยังได้รับความคุ้มครองกรณีลูกค้าสูญเสียชีวิตระหว่างปี 24 นี้ในวงเงินที่สูงถึง 384,000 บาทตั้งแต่วันที่ลูกค้าได้รับการอนุมัติกรมธรรม์ให้อีกด้วยซึ่งเงินก้อนนี้สามารถส่งต่อเป็นมรดกให้ลูกหลานหรือพ่อแม่ได้ค่ะโดยคุ้มครองทุกกรณีของการเสียชีวิตยกเว้นการฆ่าตัวตายปีแรกหรือถูกผู้รับผลประโยชน์ฆาตกรรมค่ะโครงการนี้เป็นประกันชีวิตแบบสะสมทรัพย์ที่ลูกค้าได้รับผลตอบแทนครบทุกด้านทั้งผลตอบแทนในรูปแบบเงินคืนการันตีทุกปี 24 ปีความคุ้มครองชีวิตและประโยชน์ในการลดหย่อนภาษีเราจึงจัดทำโครงการนี้เป็นพิเศษไม่เหมือนใครเพื่อคุณลูกค้าโดยเฉพาะเลยค่ะคุณลูกค้าสนใจเข้าร่วมโครงการหรือไม่คะ แล้วมันเอาไปลงยังไงอะคะมีความเสี่ยงมั้ยคะ ขายดีขอแจ้งคุณลูกค้าแบบนี้นะคะโครงการนี้ไม่ได้เป็นการลงทุนค่ะคุณลูกค้าแต่เป็นการออมเงินและได้สิทธิประโยชน์ในการคุ้มครองชีวิตไปด้วยซึ่งต่างจากการนำเงินไปลงทุนในหุ้นหรือตลาดทุนที่คุณลูกค้าอาจจะเสี่ยงที่จะขาดทุนหรือสูญเสียเงินจากการลงทุนได้ค่ะ  อ๋อ อ๋อ เข้าใจแล้วค่ะ คุณลูกค้ายืนยันเข้าร่วมโครงการนะคะขายดีขอทวนชื่อคุณลูกค้าเป็นคุณโชคดีมีเงิน อยู่บ้านเลขที่ 69/96หมู่บ้านโชคดีทวีสุข ถนนรัชดาภิเษก แขวงดินแดง เขตดินแดง กรุงเทพ 10310ถูกต้องไหมคะ ถูกต้องค่ะ ขอบคุณค่ะ คุณลูกค้าภายใน 15 วันทำการทางบริษัทจะส่งเอกสารรายละเอียดการออมให้เก็บไว้เป็นหลักฐาน โดยเป็นกรมธรรม์ฉบับสะสมทรัพย์ออมเพิ่มสุขที่จะระบุ ผลตอบแทนแบบการันตีทุกๆตัวเลข ตามที่เจ้าหน้าที่แจ้งทั้งหมด จัดส่งตามที่อยู่ที่ระบุไว้เลยไหมคะคุณลูกค้า ได้เลยค่ะ ขอบคุณค่ะคุณลูกค้า ขายดีขอขอบพระคุณคุณลูกค้าเป็นอย่างสูง และขอให้คุณลูกค้ามีสุขภาพแข็งแรงตลอดปี 2525 นะคะ สวัสดีค่ะ สวัสดีค่ะ"

In [4]:
ref_text = ''.join(ref_text.split())
ref_text

'สวัสดีค่ะขอเรียนสายคุณโชคดีมีเงินค่ะดิฉันนางสาวขายดีทวีทรัพย์ติดต่อจากบริษัทคุ้มดีจำกัดในเครือธนาคารFin4uทางบริษัทได้จัดทำโครงการพิเศษชื่อว่าคุ้มดีมีคืนสะดวกเรียนสายได้สักครู่ไหมคะสะดวกค่ะขอบพระคุณค่ะหนูขออนุญาตบันทึกสายสนทนานะคะดิฉันนางสาวขายดีทวีทรัพย์เลขที่ใบอนุญาตตัวแทนตท1902/2025ค่ะทางโครงการขอเรียนเชิญคุณลูกค้าเข้าร่วมโครงการคุ้มดีมีคืนค่ะซึ่งเป็นโครงการออมทรัพย์พิเศษในรูปแบบการประกันชีวิตสะสมทรัพย์ซึ่งผลประโยชน์ที่ลูกค้าจะได้รับแตกต่างจากที่อื่นๆเพราะการันตีผลตอบแทนรวมตลอดสัญญาสูงสุดถึง616%ของทุนประกันชีวิตเลยค่ะโดยคุณลูกค้าไม่ต้องใช้เงินก้อนเป็นหมื่นเป็นแสนเลยนะคะเพียงออมเงินทุกๆเดือนเท่าๆกันตัวอย่างเช่นคุณลูกค้าจ่ายเบี้ยเพียงวันละ90บาทหรือรายเดือนเดือนละ2,700บาทคุณลูกค้าจะได้เงินคืนทุกปีตั้งแต่ปีที่1จนถึงปีที่14การันตีรับเงินคืนปีละ3,200บาท14ปีเท่ากับได้ออมเพิ่มขึ้น44,800บาทสิ้นปีที่15-24รับเงินการันตีเพิ่มขึ้นไปอีกเป็นปีละ6,400บาทรวม10ปีเป็นเงินเพิ่ม64,000บาทค่ะและเมื่อสิ้นปีที่24ที่ครบอายุสัญญาคุณลูกค้าจะได้เงินออม384,000บาทเมื่อรวมกับเงินคืนที่ได้ไปในปีที่1ถึงปีที่24แล้วเป

# __Whisper Model__

In [9]:
# Load the Whisper model.
model = whisper.load_model("large") # large / medium / small / base / tiny

100%|█████████████████████████████████████| 2.88G/2.88G [00:41<00:00, 74.5MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [12]:
# Load the audio file.
audio_path = '/content/Telesales Call Recording 2025-02-25 08_23_26.m4a'

audio, sr = librosa.load(audio_path, sr=16000, mono=True)

<ipython-input-12-a3d1386bc44e>:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [29]:
# Save audio to a temporary .wav file for Whisper.
temp_wav = 'temp_audio.wav'
sf.write(temp_wav, audio, sr)

In [28]:
# Transcribe audio using Whisper.
%%time
result = model.transcribe(temp_wav, language='th')
transcription = result['text']

print("Transcription:", transcription)

Transcription: สวัสดีค่ะ ขอเรียนสายคุณโชคดีมีเงินค่ะดิฉัน นางสาวขายดีทวีซับ ติดต่อจากบริษัทคุ้มดีจำกัดในเครือธนาคาร Fin4uทางบริษัทได้จัดทำโครงการพิเศษ ชื่อว่า คุ้มดีมีคืนสะดวกเรียนสายได้สักครู่ไหมคะสะดวกค่ะออกพระคุณค่ะหนูขออนุญาตบันทึกสายสนทนานะคะดิฉัน นางสาวขายดีทวีซับ เลขที่ใบอนุญาต ตัวแทน ต่อท. 1902-2025 ค่ะทางโครงการขอเรียนเชิญคุณลูกค้าเข้าร่วมโครงการ คุ้มดีมีคืนค่ะซึ่งเป็นโครงการออมซับพิเศษในรูปแบบการประกันชีวิตสะสมซับซึ่งผลประโยชน์ที่ลูกค้าได้เชิญค่ะคุณลูกค้าจะได้รับแตกต่างจากที่อื่นๆเพราะการันตีผลตอบแทนรวมตลอดสัญญาสูงสุดถึง 616% ของทุนประกันชีวิตเลยค่ะโดยคุณลูกค้าไม่ต้องใช้เงินก้อนเป็นหมื่นเป็นแสนเลยนะคะเพียงออมเงินทุกๆเดือนเท่าๆกันตัวอย่างเช่น คุณลูกค้าจ่ายเบี้ยเพียงวันละ 90 บาทหรือลายเดือนเดือนละ 2,700 บาทคุณลูกค้าจะได้เงินคืนทุกปีตั้งแต่ปีที่-1 จนถึง ปีที่ 14การันตีรับเงินคืน ปีละ 3,200 บาท เท่ากับได้ออมเพิ่มขึ้น 44,800 บาทสิ้นปีที่ 15-24 รับเงินการันตีเพิ่มขึ้นไปอีกเป็นปีละ 6,400 บาทรวม 10 ปีเป็นเงินเพิ่ม 64,000 บาทค่ะและเมื่อสิ้นปีที่ 24 ที่ครบอายุสัญญาคุณลูกค้าจะได้เงินออม

In [17]:
text_large = transcription
text_large = ''.join(text_large.split())
text_large

'สวัสดีค่ะขอเรียนสายคุณโชคดีมีเงินค่ะดิฉันนางสาวขายดีทวีซับติดต่อจากบริษัทคุ้มดีจำกัดในเครือธนาคารFin4uทางบริษัทได้จัดทำโครงการพิเศษชื่อว่าคุ้มดีมีคืนสะดวกเรียนสายได้สักครู่ไหมคะสะดวกค่ะออกพระคุณค่ะหนูขออนุญาตบันทึกสายสนทนานะคะดิฉันนางสาวขายดีทวีซับเลขที่ใบอนุญาตตัวแทนต่อท.1902-2025ค่ะทางโครงการขอเรียนเชิญคุณลูกค้าเข้าร่วมโครงการคุ้มดีมีคืนค่ะซึ่งเป็นโครงการออมซับพิเศษในรูปแบบการประกันชีวิตสะสมซับซึ่งผลประโยชน์ที่ลูกค้าได้เชิญค่ะคุณลูกค้าจะได้รับแตกต่างจากที่อื่นๆเพราะการันตีผลตอบแทนรวมตลอดสัญญาสูงสุดถึง616%ของทุนประกันชีวิตเลยค่ะโดยคุณลูกค้าไม่ต้องใช้เงินก้อนเป็นหมื่นเป็นแสนเลยนะคะเพียงออมเงินทุกๆเดือนเท่าๆกันตัวอย่างเช่นคุณลูกค้าจ่ายเบี้ยเพียงวันละ90บาทหรือรายเดือนเดือนละ2,700บาทคุณลูกค้าจะได้เงินคืนทุกปีตั้งแต่ปีที่1จนถึงปีที่14การันตีรับเงินคืนปีละ3,200บาท14ปีเท่ากับได้ออมเพิ่มขึ้น44,800บาทสิ้นปีที่15-24รับเงินการันตีเพิ่มขึ้นไปอีกเป็นปีละ6,400บาทรวม10ปีเป็นเงินเพิ่ม64,000บาทและเมื่อสิ้นปีที่24ที่ครบอายุสัญญาคุณลูกค้าจะได้เงินออม384,000บาทเมื่อรวมกับเงินคืนที่ได้ไปในปีที่1ถึงปีที่2

# __Evaluation__

In [26]:
# Original text and transcribed text.
reference_text = ref_text
hypothesis_text = text_large

# Tokenize Thai text before calculating WER and CER.
reference_tokens = ' '.join(word_tokenize(reference_text, engine='newmm'))
hypothesis_tokens = ' '.join(word_tokenize(hypothesis_text, engine='newmm'))

# Calculate %WER and %CER.
wer_score = wer(reference_tokens, hypothesis_tokens) * 100
cer_score = cer(reference_text, hypothesis_text) * 100

print(f"Word Error Rate (WER): {wer_score:.2f}%")
print(f"Character Error Rate (CER): {cer_score:.2f}%")

Word Error Rate (WER): 9.91%
Character Error Rate (CER): 4.83%
